In [3]:
import pandas as pd
import os
import json
from datetime import datetime

# Chemin du dossier contenant les fichiers CSV
folder_path = r"./historique_lit"
# Chemin du fichier de suivi de la dernière exécution
last_run_file = 'last_run.json'
# Fichier de sortie Parquet
output_parquet = 'resultat_concatenation_avec_date_heure.parquet'

def load_last_run():
    try:
        with open(last_run_file, 'r') as file:
            return datetime.strptime(json.load(file)['last_run'], '%Y-%m-%d %H:%M:%S')
    except (FileNotFoundError, json.JSONDecodeError):
        return datetime.min

def save_last_run():
    with open(last_run_file, 'w') as file:
        json.dump({'last_run': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, file)

def update_dataframe(folder_path, last_run):
    dataframes = []
    # Parcourir tous les fichiers dans le dossier
    for file_name in os.listdir(folder_path):
        # Vérifier si le fichier est un CSV et modifié après la dernière exécution
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
            if file_mod_time > last_run:
                try:
                    # Lire d'abord les premières lignes pour vérifier si le fichier est valide
                    with open(file_path, 'r', encoding='cp1252', errors='replace') as f:
                        first_line = f.readline()
                        if ";" not in first_line:
                            print(f"Le fichier {file_name} ne semble pas avoir un bon format.")
                            continue
                    
                    # Lecture du fichier CSV
                    df = pd.read_csv(file_path, encoding='cp1252', sep=";", on_bad_lines='skip')
                    
                    # Vérifier que le DataFrame n'est pas vide
                    if df.empty or df.shape[1] == 1:
                        print(f"Le fichier {file_name} est vide ou mal formaté.")
                        continue
                    
                    df['PATIENT'] = df['PATIENT'].astype(str)
                    date_part = file_name[13:23]
                    time_part = file_name[24:30]
                    df['Date'] = pd.to_datetime(date_part, format='%d-%m-%Y')
                    formatted_time = f"{time_part[:2]}:{time_part[2:4]}:{time_part[4:6]}"
                    df['Heure'] = formatted_time
                    df['DateHeure'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Heure'])
                    
                    # Création de la nouvelle colonne "Localisation"
                    df['Localisation'] = df['UNITE'].astype(str) + '-' + df['CHAMBRE'].astype(str).str.strip() + '-' + df['LIT'].astype(str).str.strip()
                    
                    dataframes.append(df)
                except Exception as e:
                    print(f"Erreur avec {file_name}: {e}")
    return dataframes

# Charger la date/heure de la dernière exécution
last_run = load_last_run()
# Mettre à jour le DataFrame seulement avec les nouveaux fichiers
new_dataframes = update_dataframe(folder_path, last_run)
if new_dataframes:
    # Concaténer avec l'ancien DataFrame si le fichier Parquet existe déjà
    if os.path.exists(output_parquet):
        old_df = pd.read_parquet(output_parquet)
        final_dataframe = pd.concat([old_df] + new_dataframes, ignore_index=True)
    else:
        final_dataframe = pd.concat(new_dataframes, ignore_index=True)
    final_dataframe.to_parquet(output_parquet, index=False)
    print(f"Le fichier Parquet a été mis à jour : {output_parquet}")
else:
    print("Aucun nouveau fichier CSV détecté.")

# Sauvegarder la date/heure actuelle comme dernière exécution
save_last_run()


Aucun nouveau fichier CSV détecté.


In [4]:
final_dataframe.head()

,UNITE,CHAMBRE,LIT,PATIENT,REMARQUE,Date,Heure,DateHeure,Localisation
0,ORL,1017,1,INDISPONIBLE,TRAVAUX,NaT,NaN,NaT,NaN
1,ORL,1017,2,INDISPONIBLE,TRAVAUX,NaT,NaN,NaT,NaN
2,ABI,2031,2,INDISPONIBLE,SSE,NaT,NaN,NaT,NaN
3,ABI,2029,2,INDISPONIBLE,SSE - NON UTILISABLE,NaT,NaN,NaT,NaN
4,DIA,3067,SSE2,INDISPONIBLE,SSE OUVERTURE DMD DDG,NaT,NaN,NaT,NaN


In [5]:
df["lit_indisponible"] = final_dataframe[final_dataframe["PATIENT"] == "INDISPONIBLE"]

ValueError: Expected a 1D array, got an array with shape (12299263, 9)

In [6]:
df=final_dataframe[:100]

In [7]:
df

,UNITE,CHAMBRE,LIT,PATIENT,REMARQUE,Date,Heure,DateHeure,Localisation
0,ORL,1017,1,INDISPONIBLE,TRAVAUX,NaT,NaN,NaT,NaN
1,ORL,1017,2,INDISPONIBLE,TRAVAUX,NaT,NaN,NaT,NaN
2,ABI,2031,2,INDISPONIBLE,SSE,NaT,NaN,NaT,NaN
3,ABI,2029,2,INDISPONIBLE,SSE - NON UTILISABLE,NaT,NaN,NaT,NaN
4,DIA,3067,SSE2,INDISPONIBLE,SSE OUVERTURE DMD DDG,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
95,TOK,15,1,1011234,None,NaT,NaN,NaT,NaN
96,TOK,15,2,91005698,None,NaT,NaN,NaT,NaN
97,TOK,15,3,92023495,None,NaT,NaN,NaT,NaN
98,TOK,15,4,91009088,None,NaT,NaN,NaT,NaN


In [8]:
df["lit_indisponible"] = final_dataframe[final_dataframe["PATIENT"] == "INDISPONIBLE"]

ValueError: Expected a 1D array, got an array with shape (100, 9)

In [12]:
df["lit_indisponible"]=df["PATIENT"] == "INDISPONIBLE"

/tmp/ipykernel_2519/1018880272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lit_indisponible"]=df["PATIENT"] == "INDISPONIBLE"


In [13]:
df


,UNITE,CHAMBRE,LIT,PATIENT,REMARQUE,Date,Heure,DateHeure,Localisation,lit_indisponible
0,ORL,1017,1,INDISPONIBLE,TRAVAUX,NaT,NaN,NaT,NaN,True
1,ORL,1017,2,INDISPONIBLE,TRAVAUX,NaT,NaN,NaT,NaN,True
2,ABI,2031,2,INDISPONIBLE,SSE,NaT,NaN,NaT,NaN,True
3,ABI,2029,2,INDISPONIBLE,SSE - NON UTILISABLE,NaT,NaN,NaT,NaN,True
4,DIA,3067,SSE2,INDISPONIBLE,SSE OUVERTURE DMD DDG,NaT,NaN,NaT,NaN,True
...,...,...,...,...,...,...,...,...,...,...
95,TOK,15,1,1011234,None,NaT,NaN,NaT,NaN,False
96,TOK,15,2,91005698,None,NaT,NaN,NaT,NaN,False
97,TOK,15,3,92023495,None,NaT,NaN,NaT,NaN,False
98,TOK,15,4,91009088,None,NaT,NaN,NaT,NaN,False


In [21]:
df["n° identifiant unique"] = df["UNITE"].astype(str) + "-" + df["CHAMBRE"].astype(str)+"-"+df["LIT"].astype(str)

/tmp/ipykernel_2519/1363951620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n° identifiant unique"] = df["UNITE"].astype(str) + "-" + df["CHAMBRE"].astype(str)+"-"+df["LIT"].astype(str)


In [22]:
df

,UNITE,CHAMBRE,LIT,PATIENT,REMARQUE,Date,Heure,DateHeure,Localisation,lit_indisponible,n° identifiant unique
0,ORL,1017,1,INDISPONIBLE,TRAVAUX,NaT,NaN,NaT,NaN,True,ORL-1017 -1
1,ORL,1017,2,INDISPONIBLE,TRAVAUX,NaT,NaN,NaT,NaN,True,ORL-1017 -2
2,ABI,2031,2,INDISPONIBLE,SSE,NaT,NaN,NaT,NaN,True,ABI-2031 -2
3,ABI,2029,2,INDISPONIBLE,SSE - NON UTILISABLE,NaT,NaN,NaT,NaN,True,ABI-2029 -2
4,DIA,3067,SSE2,INDISPONIBLE,SSE OUVERTURE DMD DDG,NaT,NaN,NaT,NaN,True,DIA-3067 -SSE2
...,...,...,...,...,...,...,...,...,...,...,...
95,TOK,15,1,1011234,None,NaT,NaN,NaT,NaN,False,TOK-15 -1
96,TOK,15,2,91005698,None,NaT,NaN,NaT,NaN,False,TOK-15 -2
97,TOK,15,3,92023495,None,NaT,NaN,NaT,NaN,False,TOK-15 -3
98,TOK,15,4,91009088,None,NaT,NaN,NaT,NaN,False,TOK-15 -4
